In [1]:
import pandas as pd
import sys, os
sys.path.append(os.path.abspath(".."))

import src.preprocessing
from importlib import reload
reload(src.preprocessing)

from typing import Union
from src.preprocessing import (
    compute_cancellation,
    aggregate_user_day_activity,
    add_rolling_averages
)


In [2]:
root = '/Users/chloevacca/PycharmProjects/PyDataScience/customer churn'
df = pd.read_parquet(root + '/train.parquet')
df.head()

,status,gender,firstName,level,lastName,userId,ts,auth,page,sessionId,location,itemInSession,userAgent,method,length,song,artist,time,registration
0,200,M,Shlok,paid,Johnson,1749042,1538352001000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",278,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,524.32934,Ich mache einen Spiegel - Dream Part 4,Popol Vuh,2018-10-01 00:00:01,2018-08-08 13:22:21
992,200,M,Shlok,paid,Johnson,1749042,1538352525000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",279,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,178.02404,Monster (Album Version),Skillet,2018-10-01 00:08:45,2018-08-08 13:22:21
1360,200,M,Shlok,paid,Johnson,1749042,1538352703000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",280,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,232.61995,Seven Nation Army,The White Stripes,2018-10-01 00:11:43,2018-08-08 13:22:21
1825,200,M,Shlok,paid,Johnson,1749042,1538352935000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",281,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,265.50812,Under The Bridge (Album Version),Red Hot Chili Peppers,2018-10-01 00:15:35,2018-08-08 13:22:21
2366,200,M,Shlok,paid,Johnson,1749042,1538353200000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",282,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,471.69261,Circlesong 6,Bobby McFerrin,2018-10-01 00:20:00,2018-08-08 13:22:21


In [3]:
object_cols = df.select_dtypes(include="object").columns
df[object_cols] = df[object_cols].astype("category")
df.drop(columns=['gender', 'firstName', 'lastName', 'location', 'userAgent'], inplace=True)

In [4]:
df_new = aggregate_user_day_activity(df)
df_new['userId'] = df_new['userId'].astype(int)
df_new.head()

/Users/chloevacca/PycharmProjects/PyDataScience/kaggle-churn/src/preprocessing.py:155: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  per_day_counts = df_copy.groupby([user_col, 'date']).size().reset_index(name='event_count')
/Users/chloevacca/PycharmProjects/PyDataScience/kaggle-churn/src/preprocessing.py:157: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  session_counts = df_copy.groupby([user_col, 'date'])['sessionId'].nunique().reset_index(name='session_count')
/Users/chloevacca/PycharmProjects/PyDataScience/kaggle-churn/src/preprocessing.py:164: FutureWarning: The default of observed=False is deprecated an

,date,userId,About,Add Friend,Add to Playlist,Cancel,Downgrade,Error,Help,Home,...,Submit Upgrade,Thumbs Down,Thumbs Up,Upgrade,event_count,session_count,active_flag,events_per_session,level,days_since_registration
0,2018-10-01,1000025,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,NaN,83
1,2018-10-02,1000025,0,3,2,0,1,0,1,6,...,1,2,5,1,120,2,1,60.0,paid,84
2,2018-10-03,1000025,0,6,13,0,2,0,2,13,...,0,3,21,0,381,1,1,381.0,paid,85
3,2018-10-04,1000025,0,0,5,0,3,0,1,9,...,0,3,12,0,237,1,1,237.0,paid,86
4,2018-10-05,1000025,0,0,1,0,2,0,0,2,...,0,0,0,0,27,1,1,27.0,paid,87


In [5]:
df_new = add_rolling_averages(df_new, columns=['Add Friend', 'Add to Playlist', 'Thumbs Down', 'Thumbs Up', 'Error'], n=7)


In [6]:
denominator = df_new['thumbs_up_avg_7d'] + df_new['thumbs_down_avg_7d']
df_new['thumbs_ratio_7d'] = df_new['thumbs_up_avg_7d'] / denominator.replace(0, pd.NA)
df_new.head()

,date,userId,About,Add Friend,Add to Playlist,Cancel,Downgrade,Error,Help,Home,...,active_flag,events_per_session,level,days_since_registration,add_friend_avg_7d,add_to_playlist_avg_7d,thumbs_down_avg_7d,thumbs_up_avg_7d,error_avg_7d,thumbs_ratio_7d
0,2018-10-01,1000025,0,0,0,0,0,0,0,0,...,0,0.0,NaN,83,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-10-02,1000025,0,3,2,0,1,0,1,6,...,1,60.0,paid,84,0.00,0.0,0.000000,0.000000,0.0,<NA>
2,2018-10-03,1000025,0,6,13,0,2,0,2,13,...,1,381.0,paid,85,1.50,1.0,1.000000,2.500000,0.0,0.714286
3,2018-10-04,1000025,0,0,5,0,3,0,1,9,...,1,237.0,paid,86,3.00,5.0,1.666667,8.666667,0.0,0.83871
4,2018-10-05,1000025,0,0,1,0,2,0,0,2,...,1,27.0,paid,87,2.25,5.0,2.000000,9.500000,0.0,0.826087


In [8]:
unique_dates = sorted(df_new['date'].unique())
print(f"Computing cancellation targets for {len(unique_dates)} unique dates...")

cancellation_targets = []
df['page'] = df['page'].astype(str)

for present_date in unique_dates:
    target_df = compute_cancellation(df, present_time=present_date, window_days=10)
    target_df['date'] = present_date
    cancellation_targets.append(target_df)

target_by_date = pd.concat(cancellation_targets, ignore_index=True)
target_by_date = target_by_date.rename(columns={'userId': 'userId', 'cancellation_confirmed': 'churn_status'})

print(f"\nCancellation targets shape: {target_by_date.shape}")
print(f"Sample:")
print(target_by_date.head(10))

Computing cancellation targets for 51 unique dates...

Cancellation targets shape: (976140, 3)
Sample:
    userId  churn_status        date
0  1749042             0  2018-10-01
1  1563081             0  2018-10-01
2  1697168             0  2018-10-01
3  1222580             0  2018-10-01
4  1714398             0  2018-10-01
5  1010522             0  2018-10-01
6  1475659             0  2018-10-01
7  1558463             0  2018-10-01
8  1605667             0  2018-10-01
9  1385500             0  2018-10-01


In [43]:
# # df_new = pd.read_csv(root + '/data/df_transformed.csv')
# target_by_date = pd.read_csv(root + '/data/churn_status.csv')
#
# df_new.to_csv(root + '/data/df_transformed.csv', index=False)
# # target_by_date.to_csv(root + '/data/churn_status.csv', index=False)

In [ ]:
# Convert date column to datetime if it's not already
if df_new['date'].dtype == 'object' or str(df_new['date'].dtype).startswith('category'):
    df_new['date'] = pd.to_datetime(df_new['date'])
elif not pd.api.types.is_datetime64_any_dtype(df_new['date']):
    df_new['date'] = pd.to_datetime(df_new['date'])

# Convert df['page'] to string to avoid categorical issues in compute_cancellation
df['page'] = df['page'].astype(str)

unique_dates = sorted(df_new['date'].unique())
print(f"Computing cancellation targets for {len(unique_dates)} unique dates...")

cancellation_targets = []

for present_date in unique_dates:
    target_df = compute_cancellation(df, present_time=present_date, window_days=10)
    target_df['date'] = present_date
    cancellation_targets.append(target_df)

target_by_date = pd.concat(cancellation_targets, ignore_index=True)
target_by_date = target_by_date.rename(columns={'userId': 'userId', 'cancellation_confirmed': 'churn_status'})

print(f"\nCancellation targets shape: {target_by_date.shape}")
print(f"Sample:")
print(target_by_date.head(10))

ValueError: You are trying to merge on int64 and object columns for key 'userId'. If you wish to proceed you should use pd.concat

In [ ]:
# Split data and define variables for modeling
from sklearn.model_selection import train_test_split

# Define feature columns (exclude userId, date, and target)
feature_cols = [col for col in df_train.columns if col not in ['userId', 'date', 'churn_status']]

# Separate features and target
X = df_train[feature_cols].copy()
y = df_train['churn_status']

# Fix data types for XGBoost compatibility
# Convert 'level' category to numeric (0 for 'free', 1 for 'paid')
if 'level' in X.columns:
    X['level'] = (X['level'] == 'paid').astype(int)

# Convert thumbs_ratio_7d from object to float
if 'thumbs_ratio_7d' in X.columns:
    X['thumbs_ratio_7d'] = pd.to_numeric(X['thumbs_ratio_7d'], errors='coerce').fillna(0)

# Split into train and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")
print(f"Feature columns ({len(feature_cols)}): {feature_cols}")
print(f"\nData types after fixes:")
print(X_train.dtypes)

# For predictions on final test data
X_test_pred = X_test
df_test_final = df_train.iloc[X_test.index]


Training set size: (780912, 30)
Test set size: (195228, 30)
Feature columns (30): ['About', 'Add Friend', 'Add to Playlist', 'Cancel', 'Downgrade', 'Error', 'Help', 'Home', 'Logout', 'NextSong', 'Roll Advert', 'Save Settings', 'Settings', 'Submit Downgrade', 'Submit Upgrade', 'Thumbs Down', 'Thumbs Up', 'Upgrade', 'event_count', 'session_count', 'active_flag', 'events_per_session', 'level', 'days_since_registration', 'add_friend_avg_7d', 'add_to_playlist_avg_7d', 'thumbs_down_avg_7d', 'thumbs_up_avg_7d', 'error_avg_7d', 'thumbs_ratio_7d']

Data types after fixes:
About                        int64
Add Friend                   int64
Add to Playlist              int64
Cancel                       int64
Downgrade                    int64
Error                        int64
Help                         int64
Home                         int64
Logout                       int64
NextSong                     int64
Roll Advert                  int64
Save Settings                int64
Settings  

In [46]:
# Train Logistic Regression and generate predictions
import xgboost as xgb
import src.modeling
reload(src.modeling)
from src.modeling import (
    train_xgboost,
    evaluate_model,
    get_feature_importance,
    make_predictions,
    create_submission,
)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
max_depths = range(1, 21)
train_rmse = []
test_rmse = []
best_depth = None
best_rmse = float('inf')

for depth in range(1, 21):
    xg = XGBClassifier(max_depth=depth, random_state=42)
    xg.fit(X_train, y_train)
    rmse = np.sqrt(mean_squared_error(y_test, xg.predict(X_test)))
    
    train_rmse.append(np.sqrt(mean_squared_error(y_train, xg.predict(X_train))))
    test_rmse.append(np.sqrt(mean_squared_error(y_test, xg.predict(X_test))))
    
    if rmse < best_rmse:
        best_rmse = rmse
        best_depth = depth

print(f"Best depth: {best_depth}, RMSE: {best_rmse}")

In [ ]:
# graph for best depth

import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 6))
plt.plot(max_depths, train_rmse, label='Train RMSE', marker='o')
plt.plot(max_depths, test_rmse, label='Test RMSE', marker='s')
plt.xlabel('Max Depth')
plt.ylabel('RMSE')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Train the model

# xgb_model = train_xgboost(X_train, y_train)
# xgb_results = evaluate_model(xgb_model, X_test, y_test)
# xgb_feature_importance = get_feature_importance(xgb_model, feature_cols, top_n=10)

xgb = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=None,
    n_jobs=-1
)


xgb.fit(X_train, y_train)
# Load and preprocess test data to mirror train
print("\nLoading test data and applying preprocessing...")
df_test_raw = pd.read_parquet(root + '/test.parquet')

object_cols_test = df_test_raw.select_dtypes(include="object").columns
df_test_raw[object_cols_test] = df_test_raw[object_cols_test].astype("category")

df_test_raw = df_test_raw.drop(columns=['gender', 'firstName', 'lastName', 'location', 'userAgent'], errors='ignore')

df_test_agg = aggregate_user_day_activity(df_test_raw)
df_test_agg['userId'] = df_test_agg['userId'].astype(int)

df_test_agg = add_rolling_averages(
    df_test_agg,
    columns=['Add Friend', 'Add to Playlist', 'Thumbs Down', 'Thumbs Up', 'Error'],
    n=7
)

denominator_test = df_test_agg['thumbs_up_avg_7d'] + df_test_agg['thumbs_down_avg_7d']
df_test_agg['thumbs_ratio_7d'] = df_test_agg['thumbs_up_avg_7d'] / denominator_test.replace(0, pd.NA)

# Use the most recent date per user for prediction
df_test_latest = df_test_agg.sort_values('date').groupby('userId', as_index=False).tail(1)

# Align test features to the training feature set
X_test_submission = df_test_latest.reindex(columns=feature_cols, fill_value=0)

# Apply same data type fixes as training data
if 'level' in X_test_submission.columns:
    X_test_submission['level'] = (X_test_submission['level'] == 'paid').astype(int)

if 'thumbs_ratio_7d' in X_test_submission.columns:
    X_test_submission['thumbs_ratio_7d'] = pd.to_numeric(X_test_submission['thumbs_ratio_7d'], errors='coerce').fillna(0)

print(f"Test submission matrix shape: {X_test_submission.shape}")
print(f"Test data types: {X_test_submission.dtypes.value_counts()}")

# Predict and write submission
xgb_test_predictions, xgb_test_proba = make_predictions(xgb_model, X_test_submission)
xgb_submission_final = create_submission(
    df_test_latest['userId'].values,
    xgb_test_predictions,
    output_path=root + '/submission_xgb_logloss.csv'
)